# **Procesamiento de Lenguaje Natural**

## Maestría en Inteligencia Artificial Aplicada
#### Tecnológico de Monterrey
#### Prof Luis Eduardo Falcón Morales

### **Adtividad en Equipos Semanas 7 y 8 : LDA y LMM audio-a-texto**

* **Nombres y matrículas:**

  *   Carlos Pano Hernandez - A01066264
  *   Elemento de lista
  *   Elemento de lista

* **Número de Equipo:**


* ##### **En cada ejercicio pueden importar los paquetes o librerías que requieran.**

* ##### **En cada ejercicio pueden incluir las celdas y líneas de código que deseen.**

# **Ejercicio 1:**

* #### **Liga de los audios de las fábulas de Esopo:** https://www.gutenberg.org/ebooks/21144

* #### **Descargar los 10 archivos de audio solicitados: 1, 4, 5, 6, 14, 22, 24, 25, 26, 27.**



In [37]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...

import os
import requests
import librosa

# Define the download directory
download_dir = "data"

# Create directory if it doesn't exist
if not os.path.exists(download_dir):
    os.makedirs(download_dir)

# URLs for Aesop's Fables audio files
audio_urls = {
    "audio_1": "https://www.gutenberg.org/files/21144/mp3/21144-01.mp3",
    "audio_4": "https://www.gutenberg.org/files/21144/mp3/21144-04.mp3", 
    "audio_5": "https://www.gutenberg.org/files/21144/mp3/21144-05.mp3",
    "audio_6": "https://www.gutenberg.org/files/21144/mp3/21144-06.mp3",
    "audio_14": "https://www.gutenberg.org/files/21144/mp3/21144-14.mp3",
    "audio_22": "https://www.gutenberg.org/files/21144/mp3/21144-22.mp3",
    "audio_24": "https://www.gutenberg.org/files/21144/mp3/21144-24.mp3",
    "audio_25": "https://www.gutenberg.org/files/21144/mp3/21144-25.mp3",
    "audio_26": "https://www.gutenberg.org/files/21144/mp3/21144-26.mp3",
    "audio_27": "https://www.gutenberg.org/files/21144/mp3/21144-27.mp3"
}

# Download each audio file
for audio_name, url in audio_urls.items():
    file_path = os.path.join(download_dir, f"{audio_name}.mp3")
    
    # Only download if file doesn't exist
    if not os.path.exists(file_path):
        response = requests.get(url)
        if response.status_code == 200:
            with open(file_path, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded {audio_name}")
        else:
            print(f"Failed to download {audio_name}")
    else:
        print(f"{audio_name} already exists")
        
    # Load and print duration for each audio file
    audio, sample_rate = librosa.load(file_path)
    print(f"Duración de {audio_name}: {len(audio)/sample_rate:.1f} segundos")


audio_1 already exists
Duración de audio_1: 72.6 segundos
audio_4 already exists
Duración de audio_4: 67.1 segundos
audio_5 already exists
Duración de audio_5: 58.4 segundos
audio_6 already exists
Duración de audio_6: 71.8 segundos
audio_14 already exists
Duración de audio_14: 58.1 segundos
audio_22 already exists
Duración de audio_22: 57.2 segundos
audio_24 already exists
Duración de audio_24: 64.5 segundos
audio_25 already exists
Duración de audio_25: 56.5 segundos
audio_26 already exists
Duración de audio_26: 63.7 segundos
audio_27 already exists
Duración de audio_27: 60.3 segundos


## Cargamos credenciales de OpenAI

In [38]:
# Load environment variables from the specific .env file -> Amigos, aqui cada quien configuere su propia API key en el .env
# El .gitignore ya excluye el mismo.

from openai import OpenAI
import openai
from dotenv import load_dotenv

load_dotenv('.env')

api_key = os.getenv('OPENAI_API_KEY')
if api_key is None:
    raise ValueError("OPENAI_API_KEY environment variable is not set")

client = OpenAI(api_key=api_key)

# Prueba de la API de OpenAI
response = client.responses.create(
    model="gpt-4o-mini",
    input="Hola, como estas? (Responde en 1 palabra y en Aleman)"
)

# Imprimir la respuesta como test
print(response.output_text)

Gut.


# **Ejercicio 2a:**

* #### **Comenten el por qué del modelo seleccionado para extracción del texto de los audios.**

* #### **Extraer el contenido de los audios en texto.**

* #### **Sugerencia:** pueden extraerlo en un formato de diccionario, clave:valor $→$ {audio01:fabula01, ...}

In [39]:
import json

# Diccionario para almacenar las transcripciones
transcriptions = {}

# Crear directorio para transcripciones si no existe
transcriptions_dir = 'data-transcriptions'
if not os.path.exists(transcriptions_dir):
    os.makedirs(transcriptions_dir)

# Iteramos sobre cada archivo de audio y lo transcribimos
for filename in os.listdir('data'):
    # Extraemos el ID del audio del nombre del archivo
    audio_id = os.path.splitext(filename)[0]
    audio_path = os.path.join('data', filename)
    transcript_path = os.path.join(transcriptions_dir, f'{audio_id}.txt')
    
    # Si ya existe la transcripción, la cargamos del archivo
    if os.path.exists(transcript_path):
        with open(transcript_path, 'r', encoding='utf-8') as f:
            transcriptions[audio_id] = f.read()
            continue
    
    # Si no existe, transcribimos el audio y guardamos el resultado
    try:
        with open(audio_path, "rb") as audio_file:
            transcript = client.audio.transcriptions.create(
                model="whisper-1", # Usamos Whisper por su buen rendimiento en español
                file=audio_file,
                language="es"
            )
            transcriptions[audio_id] = transcript.text
            
            # Guardamos la transcripción en un archivo
            with open(transcript_path, 'w', encoding='utf-8') as f:
                f.write(transcript.text)
    except Exception as e:
        print(f"Error al procesar {audio_id}: {str(e)}")

# Guardamos las transcripciones en formato JSON
json_path = os.path.join(transcriptions_dir, 'transcriptions.json')
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(transcriptions, f, indent=2, ensure_ascii=False)

# Mostramos las transcripciones
print("\nTranscripciones:")
print("----------------")
for audio_id, text in transcriptions.items():
    print(f"\nAudio {audio_id}:")
    print(text)



Transcripciones:
----------------

Audio audio_27:
Las fábulas de Esopo. Grabado para LibriVox.org por Elo Chito. Fábula número 87. El perro que perseguía al león. Un perro de casa se encontró con un león y partió en su persecución. Pero el león se volvió rugiendo y el perro, todo temorizado, retrocedió rápidamente por el mismo camino. Le vio una zorra y le dijo, ¡Perro infeliz! Primero perseguías al león y ya ni siquiera soportas sus surgidos. Cuando entres a una empresa, mantente siempre listo a afrontar imprevistos que no te imaginabas. Fin de la fábula. Esta grabación es del dominio público.

Audio audio_26:
Las fábulas de Esopo. Grabado para LibriVox.org por El Ochito. Fábula número 86. El perro con campanilla. Había un perro que acostumbraba a morder sin razón. Le puso su amo una campanilla para advertirle a la gente de su presencia cercana. Y el can, sonando la campanilla, se fue a la plaza pública a presumir. Mas una sabia perra, ya avanzada de años, le dijo. ¿De qué presumes 

# **Comentario sobre el modelo seleccionado (Whisper):**
# 
El modelo Whisper fue seleccionado para la transcripción de audio por las siguientes razones:
* Tiene excelente rendimiento en español, con alta precisión en el reconocimiento de palabras y acentos
* Es un modelo multilingüe entrenado con una gran cantidad de datos en diversos idiomas
* Maneja bien diferentes acentos y variaciones del español
* Es robusto ante ruido de fondo y diferentes calidades de audio
* Está optimizado para transcripción de voz a texto, que es exactamente nuestra tarea


# **Ejercicio 2b:**

* #### **Eliminar el inicio y final comunes de los textos extraídos de cada fábula.**

* #### **Sugerencia:** Pueden guardar esta información en un archivo tipo JSON, para que al estar probando diferentes opciones en los ejercicios siguientes, puedan recuperar rápidamente la información de cada video/fábula.

In [40]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...

import re

# Patrones comunes al inicio y final de las transcripciones
inicio_patterns = [
    r"Las fábulas de (?:Esopo|Sopo)\.?\s*(?:Grabado para LibriVox\.org|grabado para Librevox\.org)\s*(?:por\s+)?",
    r"(?:por\s+)?(?:Elo Chito|Karen Savage|Karen Savich|El Ochito|Roberto Antonio Muñoz|Alejandro González Calderón|Paulino)\s*(?:www\.paulino\.info)?\s*",
    r"[Ff]ábula (?:número|numero) \d+\.?\s*",
    r"^[,\s]+",  # Eliminar comas y espacios al inicio
    r"Las fábulas de (?:Esopo|Sopo),?\s*grabado para (?:LibriVox|Librevox)\.org\s*[,\s\.]*"  # Nuevo patrón
]

final_patterns = [
    r"(?:Fin de (?:la )?fábula\.?\s*)?(?:Esta (?:es una )?grabación (?:es )?(?:del|de) dominio público\.?)?",
    r"Subtítulos realizados por la comunidad de Amara\.org",
    r"[,\s]+$",  # Eliminar comas y espacios al final
    r"Esta grabación es del dominio público\.",  # Nuevo patrón
    r"Fin de la fábula\."  # Nuevo patrón
]

def clean_text(text):
    # Eliminar el inicio común usando los patrones mejorados
    for pattern in inicio_patterns:
        text = re.sub(pattern, "", text, flags=re.IGNORECASE)
    
    # Eliminar el final común usando los patrones mejorados
    for pattern in final_patterns:
        text = re.sub(pattern + r"$", "", text, flags=re.IGNORECASE)
    
    # Limpiar puntuación y espacios
    text = re.sub(r'\.+', '.', text)  # Eliminar múltiples puntos seguidos
    text = re.sub(r'\s+', ' ', text)  # Normalizar espacios
    text = re.sub(r'^\s+|\s+$', '', text)  # Eliminar espacios al inicio y final
    text = re.sub(r'^[,\s\.]+', '', text)  # Eliminar comas, espacios y puntos al inicio
    
    # Asegurar que el texto termine con un punto
    if not text.endswith('.'):
        text = text.rstrip('.') + '.'
        
    return text

# Crear directorio para guardar las transcripciones limpias
cleaned_dir = "data-transcriptions-cleaned"
if not os.path.exists(cleaned_dir):
    os.makedirs(cleaned_dir)

# Limpiar las transcripciones y guardarlas
cleaned_transcriptions = {}
for audio_id, text in transcriptions.items():
    cleaned_text = clean_text(text)
    cleaned_transcriptions[audio_id] = cleaned_text
    
    # Guardar cada transcripción limpia individualmente
    cleaned_path = os.path.join(cleaned_dir, f'{audio_id}_cleaned.txt')
    with open(cleaned_path, 'w', encoding='utf-8') as f:
        f.write(cleaned_text)

# Guardar todas las transcripciones limpias en JSON
json_path = os.path.join(cleaned_dir, 'cleaned_transcriptions.json')
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(cleaned_transcriptions, f, ensure_ascii=False, indent=2)

# Mostrar ejemplos de las transcripciones limpias
print("\nEjemplos de transcripciones limpias:")
print("-----------------------------------")
for audio_id, text in list(cleaned_transcriptions.items())[:3]:
    print(f"\nAudio {audio_id}:")
    print(text)



Ejemplos de transcripciones limpias:
-----------------------------------

Audio audio_27:
El perro que perseguía al león. Un perro de casa se encontró con un león y partió en su persecución. Pero el león se volvió rugiendo y el perro, todo temorizado, retrocedió rápidamente por el mismo camino. Le vio una zorra y le dijo, ¡Perro infeliz! Primero perseguías al león y ya ni siquiera soportas sus surgidos. Cuando entres a una empresa, mantente siempre listo a afrontar imprevistos que no te imaginabas.

Audio audio_26:
El perro con campanilla. Había un perro que acostumbraba a morder sin razón. Le puso su amo una campanilla para advertirle a la gente de su presencia cercana. Y el can, sonando la campanilla, se fue a la plaza pública a presumir. Mas una sabia perra, ya avanzada de años, le dijo. ¿De qué presumes tanto, amigo? Sé que no llevas esa campanilla por tus grandes virtudes, sino para anunciar tu maldad oculta. Los halagos que se hacen a sí mismo, los fanfarrones, sólo delatan sus 

# **Ejercicio 3:**

* #### **Apliquen el proceso de limpieza que consideren adecuado.**

* #### **Justifiquen los pasos de limpieza utilizados. Tomen en cuenta que el texto extraído de cada fábula es relativamente pequeño.**

* #### **En caso de que decidan no aplicar esta etapa de limpieza, deberán justificarlo.**

In [41]:
# Importar las bibliotecas necesarias
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Cargar las transcripciones limpias
with open('data-transcriptions-cleaned/cleaned_transcriptions.json', 'r', encoding='utf-8') as f:
    cleaned_transcriptions = json.load(f)

# Convertir el diccionario a una lista de textos
texts = list(cleaned_transcriptions.values())

# Inicializar stemmer para español
stemmer = SnowballStemmer('spanish')

# Funciones de preprocesamiento de texto
def clean_text_es(text):
    """
    Función para limpiar y normalizar texto en español
    """
    # Convertir a minúsculas
    text = text.lower()
    
    # Eliminar signos de puntuación y números
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    
    # Tokenizar
    tokens = word_tokenize(text, language='spanish')
    
    # Eliminar stopwords
    stop_words = set(stopwords.words('spanish'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Eliminar tokens de un solo caracter
    tokens = [token for token in tokens if len(token) > 1]
    
    return tokens

def normalize_tokens_es(tokens):
    """
    Función para normalizar tokens en español usando NLTK Snowball Stemmer
    """
    normalized = []
    for token in tokens:
        # Limpiar caracteres repetidos
        token = re.sub(r'(.)\1{2,}', r'\1\1', token)
        # Aplicar stemming
        stem = stemmer.stem(token)
        normalized.append(stem)
        
    return normalized

# Aplicar limpieza y tokenización
cleaned_texts = [clean_text_es(text) for text in texts]
print("Ejemplo de tokens después de limpieza:")
print(*cleaned_texts[0:2], sep='\n\n')

# Aplicar normalización
normalized_texts = [normalize_tokens_es(text) for text in cleaned_texts]
print("\nEjemplo de tokens después de normalización:")
print(*normalized_texts[0:2], sep='\n\n')


Ejemplo de tokens después de limpieza:
['perro', 'perseguía', 'león', 'perro', 'casa', 'encontró', 'león', 'partió', 'persecución', 'león', 'volvió', 'rugiendo', 'perro', 'temorizado', 'retrocedió', 'rápidamente', 'mismo', 'camino', 'vio', 'zorra', 'dijo', 'perro', 'infeliz', 'primero', 'perseguías', 'león', 'siquiera', 'soportas', 'surgidos', 'entres', 'empresa', 'mantente', 'siempre', 'listo', 'afrontar', 'imprevistos', 'imaginabas']

['perro', 'campanilla', 'perro', 'acostumbraba', 'morder', 'razón', 'puso', 'amo', 'campanilla', 'advertirle', 'gente', 'presencia', 'cercana', 'can', 'sonando', 'campanilla', 'plaza', 'pública', 'presumir', 'mas', 'sabia', 'perra', 'avanzada', 'años', 'dijo', 'presumes', 'amigo', 'sé', 'llevas', 'campanilla', 'grandes', 'virtudes', 'sino', 'anunciar', 'maldad', 'oculta', 'halagos', 'hacen', 'mismo', 'fanfarrones', 'sólo', 'delatan', 'mayores', 'defectos', 'fin', 'fábula']

Ejemplo de tokens después de normalización:
['perr', 'persegu', 'leon', 'perr', 

# **Ejercicio 4:**
De cada fábula deberán extraer las palabras clave mediante el algoritmo LDA
(Asignación Latente de Dirichlet). Consideren que cada fábula tiene solo un tópico y
que el total de palabras por tópico serían en principio 20 (pero pueden ajustar este
valor si consideran que obtienen un mejor resultado). Deberán desplegar las
palabras clave obtenidas en cada fábula.

In [45]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...


# Importar bibliotecas necesarias para LDA
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# Convertir tokens normalizados de vuelta a texto para usar con CountVectorizer
normalized_docs = [' '.join(tokens) for tokens in normalized_texts]

# Crear vectorizador y matriz documento-término
vectorizer = CountVectorizer(max_features=1000)
doc_term_matrix = vectorizer.fit_transform(normalized_docs)

# Obtener lista de términos
terms = vectorizer.get_feature_names_out()

# Definir parámetros
num_words = 20  # Palabras clave por tópico
num_docs = len(normalized_docs)  # Número de fábulas

# Analizar cada fábula individualmente
for i in range(num_docs):
    print(f"\nPalabras clave para la fábula {i+1}:")
    
    # Crear matriz documento-término para la fábula individual
    doc_matrix = vectorizer.transform([normalized_docs[i]])
    
    # Entrenar modelo LDA
    lda = LatentDirichletAllocation(
        n_components=1,
        random_state=42,
        max_iter=15
    )
    lda.fit(doc_matrix)
    
    # Obtener las palabras más importantes
    topic = lda.components_[0]
    top_words_idx = topic.argsort()[-num_words:][::-1]
    top_words = [terms[idx] for idx in top_words_idx]
    
    # Imprimir palabras clave
    print(", ".join(top_words))





Palabras clave para la fábula 1:
leon, perr, persegu, dij, list, entres, siqu, siempr, encontr, empres, mantent, camin, surg, rug, mism, retroced, cas, rapid, part, primer

Palabras clave para la fábula 2:
campanill, perr, presum, ocult, años, se, sabi, can, cercan, razon, pus, public, presenci, plaz, defect, delat, sin, mord, mism, mayor

Palabras clave para la fábula 3:
reflej, llev, perr, rio, ajen, propi, pedaz, carn, troz, mayor, esfuerz, arrebat, result, solt, sol, codic, supuest, pued, pues, nunc

Palabras clave para la fábula 4:
carnicer, perr, accident, sal, carnic, client, cog, com, corr, pod, pens, penetr, oye, dej, ocup, not, mirart, encuentr, esper, evit

Palabras clave para la fábula 5:
huev, perr, lueg, almej, dij, asunt, sint, entrañ, entrar, mal, dos, merec, dificultad, desgarr, nunc, reflexion, redond, com, creyend, pens

Palabras clave para la fábula 6:
ceb, lob, com, caball, dej, domini, llev, oir, public, oid, coment, creersel, dient, aunqu, camin, mastic, buen, c

# **Ejercicio 5a y 5b:**

* #### **5a: Mediante el LLM que hayan seleccionado, generar un único enunciado que describa o resuma cada fábula.**

* #### **5b: Mediante el LLM que hayan seleccionado, generar tres posibles enunciados diferentes relacionados con la historia de la fábula.**

* #### **Sugerencia:** En realidad los dos incisos a y b se pueden obtener con un solo prompt que solicite la información y el formato correspondiente para cada una de estas partes. Por ejemplo, para cada fábula la salida puede ser un primer enunciado genérico que resume o describe dicha temática; seguido de tres enunciados, cada uno hablando sobre una situación o parte diferente de la fábula.

In [51]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...


# Diccionario para almacenar los resultados y evitar llamadas repetidas a la API
cached_results = {}

# Función para generar resumen y subtemas usando GPT-4
def generate_summary_and_subtopics(keywords):
    # Crear una clave única basada en las palabras clave
    cache_key = ','.join(sorted(keywords))
    
    # Verificar si ya tenemos el resultado en caché
    if cache_key in cached_results:
        return cached_results[cache_key]
    
    prompt = f"""
    Basándote en las siguientes palabras clave de una fábula de Esopo:
    {', '.join(keywords)}
    
    Por favor genera:
    1. Un único enunciado que resuma o describa la fábula
    2. Tres subtemas o enunciados diferentes relacionados con la historia
    
    Formato de respuesta:
    Resumen: [enunciado]
    Subtemas:
    - [subtema 1]
    - [subtema 2] 
    - [subtema 3]
    """
    
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=300
        )
        result = response.choices[0].message.content
        
        # Guardar el resultado en caché
        cached_results[cache_key] = result
        return result
    except Exception as e:
        return f"Error al generar resumen: {str(e)}"

# Diccionario para almacenar los resultados de cada fábula
fabulas_resultados = {}

# Generar resúmenes y subtemas para cada fábula
print("\nGenerando resúmenes y subtemas para cada fábula usando GPT-4:\n")
print("-" * 80)

for i in range(num_docs):
    print(f"\nFábula {i+1}:")
    
    # Crear matriz documento-término para la fábula individual
    doc_matrix = vectorizer.transform([normalized_docs[i]])
    
    # Entrenar modelo LDA
    lda = LatentDirichletAllocation(n_components=1, random_state=42, max_iter=15)
    lda.fit(doc_matrix)
    
    # Obtener palabras clave
    top_words_idx = lda.components_[0].argsort()[-num_words:][::-1]
    keywords = [terms[idx] for idx in top_words_idx]
    
    print("Palabras clave:", ", ".join(keywords))
    
    # Generar resumen y subtemas
    result = generate_summary_and_subtopics(keywords)
    print("\nAnálisis GPT-4:")
    print(result)
    print("-" * 80)
    
    # Guardar resultados de esta fábula
    fabulas_resultados[f'fabula_{i+1}'] = {
        'palabras_clave': keywords,
        'analisis': result
    }

# Guardar todos los resultados en un archivo JSON
import json
with open('fabulas_analisis.json', 'w', encoding='utf-8') as f:
    json.dump({
        'cached_results': cached_results,
        'fabulas_resultados': fabulas_resultados
    }, f, ensure_ascii=False, indent=2)





Generando resúmenes y subtemas para cada fábula usando GPT-4:

--------------------------------------------------------------------------------

Fábula 1:
Palabras clave: leon, perr, persegu, dij, list, entres, siqu, siempr, encontr, empres, mantent, camin, surg, rug, mism, retroced, cas, rapid, part, primer

Análisis GPT-4:
Resumen: La fábula relata la historia de un león que persigue a un perro listo, quien siempre encuentra maneras ingeniosas para mantenerse un paso por delante del león, demostrando que la mente rápida puede superar la fuerza bruta.

Subtemas:
- La inteligencia y astucia del perro para evadir al león.
- El persistente intento del león por atrapar al perro.
- La importancia de la agilidad mental frente a la fuerza física en situaciones de peligro.
--------------------------------------------------------------------------------

Fábula 2:
Palabras clave: campanill, perr, presum, ocult, años, se, sabi, can, cercan, razon, pus, public, presenci, plaz, defect, delat, si

# **Ejercicio 6:**

* #### **Incluyan sus conclusiones de la actividad audio-a-texto:**



None

# **Fin de la actividad LDA y LMM: audio-a-texto**